In [25]:
import os
import SimpleITK as sitk
import pandas as pd
from radiomics import featureextractor

# Define paths
path_to_data_t1 = "/Intern Programming/data/UMCGOPC_MRI_nii_zscoreinHNmask"
path_to_data_t1gt0tt = "/Intern Programming/data/output/t1t2gtv_1mm"
clc_data_path = '/Intern Programming/data/OPCdigits_333_split_t1good.csv'
clc_data = pd.read_csv(clc_data_path).drop(columns=['Unnamed: 0'])

# Get list of CT images and GTV masks
patients_number = 333
files = clc_data['PatientID']
mri_images = [os.path.join(path_to_data_t1, f'{x}_t1.nii.gz') for x in files]
gtv_masks = [os.path.join(path_to_data_t1gt0tt, f'{x}_gtvpttot1.nii.gz') for x in files]

# Create Radiomics feature extractor object
extractor = featureextractor.RadiomicsFeatureExtractor()

# Enable desired feature classes
extractor.enableAllFeatures()  # Enable all feature classes
extractor.enableFeatureClassByName('wavelet')
extractor.enableFeatureClassByName('LoG')

# Adjust settings
extractor.settings['binWidth'] = 25  # Adjust intensity discretization
extractor.settings['resampledPixelSpacing'] = (0.5, 0.5, 3.0)  # Resampling pixel spacing
extractor.settings['interpolator'] = 'sitkLinear'  # Interpolator for image
extractor.settings['resamplingInterpolator'] = 'sitkBSpline'  # Interpolator for resampling
extractor.settings['correctMask'] = True  # Correct mask for features

# Initialize list to store feature DataFrames
feature_dfs = []

# Iterate over patients and extract features
for patient_id, ct_path, mask_path in zip(files, mri_images, gtv_masks):
    ct_image = sitk.ReadImage(ct_path)
    mask_image = sitk.ReadImage(mask_path)
    
    try:
        # Check mask dimensionality and adjust extractor settings accordingly
        if mask_image.GetDimension() > 2:
            extractor.settings['force2D'] = False  # Allow 3D extraction if mask is 3D
        else:
            extractor.settings['force2D'] = True  # Force 2D extraction for shape features
        
        features = extractor.execute(ct_image, mask_image)
        features['PatientID'] = patient_id
        features_df = pd.DataFrame([features])  # Create DataFrame with extracted features
        feature_dfs.append(features_df)
    except Exception as e:
        print(f"Error processing {patient_id}: {e}")

# Concatenate all feature DataFrames into a single DataFrame
if feature_dfs:
    all_features_df = pd.concat(feature_dfs, ignore_index=True)
    # Save or further process all_features_df as needed
    print(all_features_df.head())
else:
    print("No features extracted.")


Feature class wavelet is not recognized
Feature class LoG is not recognized


parameter force2D must be set to True to enable shape2D extraction
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
parameter force2D must be set to True to enable shape2D extraction
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


KeyboardInterrupt: 

In [23]:
df = pd.DataFrame(all_features_df)
df.columns

df.to_csv(r'/Intern Programming/data/output/' + 'umcg_radiomicsfeature_t1_GTVtot(1).csv' )

## geometric, texture and intensity (study these things)